In [1]:
import numpy as np
from tqdm import tqdm
import requests

In [66]:
index = 'test1'
samples = 100
batch_size = 10000

In [3]:
def create_random_point(x0,y0,distance):

    """
    Utility method for simulation of the points
    """   
    r = distance / 111300
    u = np.random.uniform(0,1)
    v = np.random.uniform(0,1)
    w = r * np.sqrt(u)
    t = 2 * np.pi * v
    x = w * np.cos(t)
    x1 = x / np.cos(y0)
    y = w * np.sin(t)
    return (np.radians(x0 + x1), np.radians(y0 + y))
        

def create_random_points_around_loc(max_elements,lat1,long1,distance_in_km):

    list_of_points= list(tuple())
    for x in range(0,max_elements): 
        latitude2,longitude2 =create_random_point(lat1,long1,distance_in_km*1000)
        list_of_points.append((latitude2,longitude2,x))
    return list_of_points

In [67]:
# Cyberjaya, Malaysia
lat, long = 2.950815010581982, 101.62843052319319
lat1, long1 = create_random_point(lat, long, 10)
random_points = create_random_points_around_loc(samples, lat, long, 10)

In [68]:
random_points[0]

(0.04967605627257294, 1.773975403061679, 0)

In [69]:
!rm -rf db/{index}
!rm -rf db/{index}-index

In [70]:
for i in tqdm(range(0, len(random_points), batch_size)):
    b = random_points[i: i + batch_size]
    r = []
    for k in range(len(b)):
        r.append({'lat': b[k][0], 'long': b[k][1], 'no': b[k][2]})
    requests.post(f'http://localhost:8080/{index}/insert', json = r)

100%|██████████| 1/1 [00:00<00:00, 23.39it/s]


In [74]:
%%time

!curl --request GET 'http://localhost:8080/test1/query?min_distance=2.0&max_distance=3.4&lat=2.950815010581982&long=101.62843052319319'

[{"distance":2.7435444445887867,"lat":0.051198664647091295,"long":1.774057179420072,"no":64},{"distance":2.908191238826303,"lat":0.05152282827772343,"long":1.7732942780718577,"no":66},{"distance":2.9317181243178023,"lat":0.05193458662198576,"long":1.7739059018276264,"no":79}]CPU times: user 4.09 ms, sys: 7.25 ms, total: 11.3 ms
Wall time: 201 ms
